In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import f1_score, jaccard_score, precision_score, recall_score, accuracy_score
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, VGG19, MobileNetV2, ResNet50
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Concatenate, MaxPool2D, BatchNormalization, Activation, UpSampling2D, Reshape
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping, TensorBoard
import albumentations as A

In [ ]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
create_dir("files")

create_dir("results")

In [ ]:
# Constants
H, W = 256, 256

# Data Augmentation with Albumentations
def augment_data(image, mask, augment=True):
    if augment:
        transform = A.Compose([
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.5),
            A.Rotate(limit=20, p=0.5),
            A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=20, p=0.5)
        ])
        augmented = transform(image=image, mask=mask)
        image = augmented['image']
        mask = augmented['mask']
    return image, mask

# TensorFlow Dataset Function
def tf_dataset(X, Y, batch_size, augment=False):
    def map_func(x, y):
        def _parse(x, y):
            x = x.decode()
            y = y.decode()
            x = read_image(x)
            y = read_mask(y)
            x, y = augment_data(x, y, augment)
            return x, y

        x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
        x.set_shape([H, W, 3])
        y.set_shape([H, W, 1])
        return x, y

    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(map_func)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(10)
    return dataset


In [ ]:
# Image and mask reading functions
def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x / 255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W, H))
    x = x / 255.0
    x = np.expand_dims(x, axis=-1)
    x = x.astype(np.float32)
    return x

# Dataset loading and splitting function
def load_dataset(path, val_split=0.2):  # Changed split to 20% for validation
    # Load training images and masks
    train_images = sorted(glob(os.path.join(path, "trainval-image", "*.jpg")))
    train_masks = sorted(glob(os.path.join(path, "trainval-mask", "*.jpg")))

    # Load test images and masks
    test_images = sorted(glob(os.path.join(path, "test-image", "*.jpg")))
    test_masks = sorted(glob(os.path.join(path, "test-mask", "*.jpg")))

    # Split training data into training and validation sets
    train_x, valid_x = train_test_split(train_images, test_size=val_split, random_state=42)
    train_y, valid_y = train_test_split(train_masks, test_size=val_split, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_images, test_masks)

# Load Dataset
dataset_path = "/kaggle/input/nlt3kdata"
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(dataset_path)
print(f"Train: ({len(train_x)}, {len(train_y)})")
print(f"Valid: ({len(valid_x)}, {len(valid_y)})")
print(f"Test: ({len(test_x)}, {len(test_y)})")

# Create Datasets
train_dataset = tf_dataset(train_x, train_y, batch_size=16, augment=True)
valid_dataset = tf_dataset(valid_x, valid_y, batch_size=16)
test_dataset = tf_dataset(test_x, test_y, batch_size=16)

In [ ]:
import matplotlib.pyplot as plt

# Number of images in each set
num_train = len(train_x)
num_valid = len(valid_x)
num_test = len(test_x)

# Set names
sets = ['Train', 'Validation', 'Test']

# Number of images in each set
counts = [num_train, num_valid, num_test]

# Create a bar chart
plt.figure(figsize=(10, 6))
plt.bar(sets, counts, color=['blue', 'green', 'red'])
plt.title('Number of Images in Each Dataset Split')
plt.xlabel('Dataset Split')
plt.ylabel('Number of Images')
plt.ylim(0, max(counts) + 10)  # Set y-axis limit slightly higher than the max count

# Add text labels for each bar
for i in range(len(sets)):
    plt.text(i, counts[i] + 1, str(counts[i]), ha = 'center', color='black')

# Show the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Number of images in each set
num_train = len(train_x)
num_valid = len(valid_x)
num_test = len(test_x)

# Set names
sets = ['Train', 'Validation', 'Test']

# Number of images in each set
counts = [num_train, num_valid, num_test]

# Create a pie chart
plt.figure(figsize=(8, 8))
plt.pie(counts, labels=sets, autopct='%1.1f%%', startangle=140, colors=['skyblue', 'lightgreen', 'lightcoral'])
plt.title('Distribution of Images in Dataset Splits')

# Show the plot
plt.show()


In [ ]:
# Model parameters
lr = 1e-4  # Lower learning rate
num_epochs = 500  # Increase number of epochs

batch_size = 16
#lr = 1e-4
#num_epochs = 300
model_path = os.path.join("files", "model.h5")
csv_path = os.path.join("files", "log.csv")
dataset_path="/kaggle/input/tn3k-thyroid-nodule-region-segmentation-dataset"

callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
    CSVLogger(csv_path),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False),
]

from tensorflow.keras.backend import sum as K_sum

def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true_flat = K.flatten(y_true)
    y_pred_flat = K.flatten(y_pred)
    intersection = K_sum(y_true_flat * y_pred_flat)
    return (2. * intersection + smooth) / (K_sum(y_true_flat) + K_sum(y_pred_flat) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)


def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = K.sum(K.flatten(y_true) * K.flatten(y_pred))
        union = K.sum(K.flatten(y_true)) + K.sum(K.flatten(y_pred)) - intersection
        return (intersection + K.epsilon()) / (union + K.epsilon())
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

In [ ]:
# Conv Block
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

# Encoder Block
def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

# Decoder Block
def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, 2, strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x


In [ ]:
# Build U-Net
def unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model

# Initialize the model
unet_model = unet((H, W, 3))

# Compile the model with the IoU metric, dice coefficient loss, and optimizer
unet_model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coef, 'accuracy', iou])

# Print the model summary to check the final model architecture
unet_model.summary()

# Train the model using the same datasets and callbacks you've defined
history_unet = unet_model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    callbacks=callbacks
)

# Save training history
np.save('history_unet.npy', history_unet.history)

# Evaluate the model on the test set and display test metrics
test_metrics = unet_model.evaluate(test_dataset)
print(f"Unet Test Loss: {test_metrics[0]}")
print(f"Unet Test Dice Coefficient: {test_metrics[1]}")
print(f"Unet Test Accuracy: {test_metrics[2]}")
print(f"Unet Test IoU: {test_metrics[3]}")

# Save the test metrics to a file
with open('unet_test_metrics.txt', 'w') as file:
    file.write(str(test_metrics))


In [ ]:
import pandas as pd
metrics = pd.read_csv("/kaggle/working/files/log.csv")
metrics.tail(2)

In [ ]:
metrics[['dice_coef','val_dice_coef']].plot()

In [ ]:
metrics[['accuracy','val_accuracy']].plot()

In [ ]:
metrics[['loss','val_loss']].plot()

In [ ]:
from tqdm import tqdm
def save_results(image, mask, y_pred, save_image_path):
    mask = np.expand_dims(mask, axis=-1)
    mask = np.concatenate([mask, mask, mask], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)
    y_pred = y_pred * 255

    line = np.ones((H, 10, 3)) * 255

    cat_images = np.concatenate([image, line, mask, line, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)
    
    """Prediction and Evaluation"""
SCORE = []
for x, y in tqdm(zip(test_x, test_y), total=len(test_y)):
    """ Extracting the name """
    name = x.split("/")[-1]

    """ Reading the image """
    image = cv2.imread(x, cv2.IMREAD_COLOR) ## [H, w, 3]
    image = cv2.resize(image, (W, H))       ## [H, w, 3]
    x = image/255.0                         ## [H, w, 3]
    x = np.expand_dims(x, axis=0)           ## [1, H, w, 3]

    """ Reading the mask """
    mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (W, H))

    """ Prediction """
    y_pred = unet_model.predict(x, verbose=0)[0]
    y_pred = np.squeeze(y_pred, axis=-1)
    y_pred = y_pred >= 0.5
    y_pred = y_pred.astype(np.int32)

    """ Saving the prediction """
    save_image_path = os.path.join("results", name)
    save_results(image, mask, y_pred, save_image_path)

    """ Flatten the array """
    mask = mask/255.0
    mask = (mask > 0.5).astype(np.int32).flatten()
    y_pred = y_pred.flatten()

    """ Calculating the metrics values """
    accuracy = accuracy_score(mask, y_pred)
    #print(accuracy)
    f1_value = f1_score(mask, y_pred, labels=[0, 1], average="binary")
    jac_value = jaccard_score(mask, y_pred, labels=[0, 1], average="binary")
    recall_value = recall_score(mask, y_pred, labels=[0, 1], average="binary", zero_division=0)
    precision_value = precision_score(mask, y_pred, labels=[0, 1], average="binary", zero_division=0)
    SCORE.append([name, accuracy, f1_value, jac_value, recall_value, precision_value])

""" Metrics values """
score = [s[1:]for s in SCORE]
score = np.mean(score, axis=0)
print(f"Accuracy: {score[0]:0.5f}")
print(f"F1: {score[1]:0.5f}")
print(f"Jaccard: {score[2]:0.5f}")
print(f"Recall: {score[3]:0.5f}")
print(f"Precision: {score[4]:0.5f}")

df = pd.DataFrame(SCORE, columns=["Image","Accuracy", "F1", "Jaccard", "Recall", "Precision"])
df.to_csv("files/score.csv", index=None)


In [ ]:
scores = pd.read_csv("/kaggle/working/files/score.csv")
scores.head()

In [ ]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

create_dir("files_vgg16")
create_dir("results_vgg16")

model_path_vgg16 = os.path.join("files_vgg16", "model_vgg16.h5")
csv_path_vgg16 = os.path.join("files_vgg16", "log_vgg16.csv")

callbacks_vgg16 = [
    ModelCheckpoint(model_path_vgg16, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
    CSVLogger(csv_path_vgg16),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False),
]

In [ ]:
# Define the U-Net model with VGG16 encoder
def build_vgg16(input_shape):
    # Encoder: load VGG16 with pre-trained ImageNet weights
    encoder = VGG16(include_top=False, weights='imagenet', input_tensor=Input(shape=input_shape))

    # Set the encoder layers to not be trainable
    for layer in encoder.layers:
        layer.trainable = False

    # Skip connections from specific layers in VGG16
    skip_connections = ['block1_conv2', 'block2_conv2', 'block3_conv3', 'block4_conv3']
    encoder_outputs = [encoder.get_layer(name).output for name in skip_connections]

    # Decoder
    x = encoder.output
    for i in range(len(encoder_outputs)):
        num_filters = 512 // (2 ** i)
        x = Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(x)
        skip_output = encoder_outputs[-(i + 1)]
        skip_output = tf.image.resize(skip_output, tf.shape(x)[1:3], method='nearest')
        x = Concatenate()([x, skip_output])
        x = conv_block(x, num_filters)

    # Final Conv2DTranspose layer to upscale to the original image size
    x = Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(x)
    outputs = Conv2D(1, (1, 1), activation='sigmoid', padding='same')(x)

    # Create the model
    model = Model(inputs=encoder.input, outputs=outputs)
    return model

# Initialize the model
vgg16_model = build_vgg16((H, W, 3))

# Compile the model with the IoU metric, dice coefficient loss, and optimizer
vgg16_model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coef, 'accuracy', iou])

# Print the model summary to check the final model architecture
vgg16_model.summary()


# Train the VGG16 U-Net model using the defined callbacks for VGG16
history_vgg16 = vgg16_model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    callbacks=callbacks_vgg16  # Using callbacks_vgg16 here
)

# Save training history
np.save('history_vgg16.npy', history_vgg16.history)


# Evaluate the model on the test set and display test metrics
vgg16_test_metrics = vgg16_model.evaluate(test_dataset)
print(f"VGG16 Test Loss: {vgg16_test_metrics[0]}")
print(f"VGG16 Test Dice Coefficient: {vgg16_test_metrics[1]}")
print(f"VGG16 Test Accuracy: {vgg16_test_metrics[2]}")
print(f"VGG16 Test IoU: {vgg16_test_metrics[3]}")

# Save training history
np.save('history_vgg16.npy', history_vgg16.history)

# Save the test metrics to a file
with open('vgg16_test_metrics.txt', 'w') as file:
    file.write(str(vgg16_test_metrics))

In [ ]:
# Evaluate the model on the test set and display test metrics
vgg16_test_metrics = vgg16_model.evaluate(test_dataset)
print(f"VGG16 Test Loss: {vgg16_test_metrics[0]}")
print(f"VGG16 Test Dice Coefficient: {vgg16_test_metrics[1]}")
print(f"VGG16 Test Accuracy: {vgg16_test_metrics[2]}")
print(f"VGG16 Test IoU: {vgg16_test_metrics[3]}")

# Save training history
np.save('history_vgg16.npy', history_vgg16.history)

# Save the test metrics to a file
with open('vgg16_test_metrics.txt', 'w') as file:
    file.write(str(vgg16_test_metrics))

In [ ]:
   """Prediction and Evaluation"""
SCORE_vgg16 = []
for x, y in tqdm(zip(test_x, test_y), total=len(test_y)):
    """ Extracting the name """
    name = x.split("/")[-1]

    """ Reading the image """
    image = cv2.imread(x, cv2.IMREAD_COLOR) ## [H, w, 3]
    image = cv2.resize(image, (W, H))       ## [H, w, 3]
    x = image/255.0                         ## [H, w, 3]
    x = np.expand_dims(x, axis=0)           ## [1, H, w, 3]

    """ Reading the mask """
    mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (W, H))

    """ Prediction """
    y_pred = vgg16_model.predict(x, verbose=0)[0]
    y_pred = np.squeeze(y_pred, axis=-1)
    y_pred = y_pred >= 0.5
    y_pred = y_pred.astype(np.int32)

    """ Saving the prediction """
    save_image_path = os.path.join("results_vgg16", name)
    save_results(image, mask, y_pred, save_image_path)

    """ Flatten the array """
    mask = mask/255.0
    mask = (mask > 0.5).astype(np.int32).flatten()
    y_pred = y_pred.flatten()

    """ Calculating the metrics values """
    accuracy = accuracy_score(mask, y_pred)
    f1_value = f1_score(mask, y_pred, labels=[0, 1], average="binary")
    jac_value = jaccard_score(mask, y_pred, labels=[0, 1], average="binary")
    recall_value = recall_score(mask, y_pred, labels=[0, 1], average="binary", zero_division=0)
    precision_value = precision_score(mask, y_pred, labels=[0, 1], average="binary", zero_division=0)
    SCORE_vgg16.append([name, accuracy, f1_value, jac_value, recall_value, precision_value])

""" Metrics values """
score_vgg16 = [s[1:]for s in SCORE_vgg16]
score_vgg16 = np.mean(score_vgg16, axis=0)
print(f"Accuracy: {score_vgg16[0]:0.5f}")
print(f"F1: {score_vgg16[1]:0.5f}")
print(f"Jaccard: {score_vgg16[2]:0.5f}")
print(f"Recall: {score_vgg16[3]:0.5f}")
print(f"Precision: {score_vgg16[4]:0.5f}")

df = pd.DataFrame(SCORE_vgg16, columns=["Image", "Accuracy","F1", "Jaccard", "Recall", "Precision"])
df.to_csv("files/score_vgg16.csv", index=None)

In [ ]:
scores = pd.read_csv("/kaggle/working/files/score_vgg16.csv")
scores.head(3)

In [ ]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

create_dir("files_vgg19")
create_dir("results_vgg19")

model_path_vgg19 = os.path.join("files_vgg19", "model_vgg19.h5")
csv_path_vgg19 = os.path.join("files_vgg19", "log_vgg19.csv")

callbacks_vgg19 = [
    ModelCheckpoint(model_path_vgg19, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
    CSVLogger(csv_path_vgg19),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False),
]

In [ ]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Concatenate, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import os

# Define the U-Net model with VGG19 encoder
def build_vgg19(input_shape):
    # Encoder: load VGG19 with pre-trained ImageNet weights
    encoder = VGG19(include_top=False, weights='imagenet', input_tensor=Input(shape=input_shape))

    # Set the encoder layers to not be trainable
    for layer in encoder.layers:
        layer.trainable = False

    # Skip connections from specific layers in VGG19
    skip_connections = ['block1_conv2', 'block2_conv2', 'block3_conv4', 'block4_conv4']
    encoder_outputs = [encoder.get_layer(name).output for name in skip_connections]

    # Decoder
    x = encoder.output
    for i in range(len(encoder_outputs)):
        num_filters = 512 // (2 ** i)
        x = Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(x)
        skip_output = encoder_outputs[-(i + 1)]
        skip_output = tf.image.resize(skip_output, tf.shape(x)[1:3], method='nearest')
        x = Concatenate()([x, skip_output])
        x = conv_block(x, num_filters)

    # Final Conv2DTranspose layer to upscale to the original image size
    x = Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(x)
    outputs = Conv2D(1, (1, 1), activation='sigmoid', padding='same')(x)

    # Create the model
    model = Model(inputs=encoder.input, outputs=outputs)
    return model

# Initialize the model
vgg19_model = build_vgg19((H, W, 3))

# Compile the model with the IoU metric, dice coefficient loss, and optimizer
vgg19_model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coef, 'accuracy', iou])

# Print the model summary to check the final model architecture
vgg19_model.summary()

# Train the VGG19 U-Net model
history_vgg19 = vgg19_model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    callbacks=callbacks_vgg19  # Make sure to define callbacks_vgg19
)

# Save training history
np.save('history_vgg19.npy', history_vgg19.history)

# Evaluate the model on the test set and display test metrics
vgg19_test_metrics = vgg19_model.evaluate(test_dataset)
print(f"VGG19 Test Loss: {vgg19_test_metrics[0]}")
print(f"VGG19 Test Dice Coefficient: {vgg19_test_metrics[1]}")
print(f"VGG19 Test Accuracy: {vgg19_test_metrics[2]}")
print(f"VGG19 Test IoU: {vgg19_test_metrics[3]}")

# Save the test metrics to a file
with open('vgg19_test_metrics.txt', 'w') as file:
    file.write(str(vgg19_test_metrics))

In [ ]:
  """Prediction and Evaluation"""
SCORE_vgg19= []
for x, y in tqdm(zip(test_x, test_y), total=len(test_y)):
    """ Extracting the name """
    name = x.split("/")[-1]

    """ Reading the image """
    image = cv2.imread(x, cv2.IMREAD_COLOR) ## [H, w, 3]
    image = cv2.resize(image, (W, H))       ## [H, w, 3]
    x = image/255.0                         ## [H, w, 3]
    x = np.expand_dims(x, axis=0)           ## [1, H, w, 3]

    """ Reading the mask """
    mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (W, H))

    """ Prediction """
    y_pred = vgg19_model.predict(x, verbose=0)[0]
    y_pred = np.squeeze(y_pred, axis=-1)
    y_pred = y_pred >= 0.5
    y_pred = y_pred.astype(np.int32)

    """ Saving the prediction """
    save_image_path = os.path.join("results_vgg19", name)
    save_results(image, mask, y_pred, save_image_path)

    """ Flatten the array """
    mask = mask/255.0
    mask = (mask > 0.5).astype(np.int32).flatten()
    y_pred = y_pred.flatten()

    """ Calculating the metrics values """
    accuracy = accuracy_score(mask, y_pred)
    f1_value = f1_score(mask, y_pred, labels=[0, 1], average="binary")
    jac_value = jaccard_score(mask, y_pred, labels=[0, 1], average="binary")
    recall_value = recall_score(mask, y_pred, labels=[0, 1], average="binary", zero_division=0)
    precision_value = precision_score(mask, y_pred, labels=[0, 1], average="binary", zero_division=0)
    SCORE_vgg19.append([name, accuracy, f1_value, jac_value, recall_value, precision_value])

""" Metrics values """
score_vgg19 = [s[1:]for s in SCORE_vgg19]
score_vgg19 = np.mean(score_vgg19, axis=0)
print(f"Accuracy: {score_vgg19[0]:0.5f}")
print(f"F1: {score_vgg19[1]:0.5f}")
print(f"Jaccard: {score_vgg19[2]:0.5f}")
print(f"Recall: {score_vgg19[3]:0.5f}")
print(f"Precision: {score_vgg19[4]:0.5f}")

df = pd.DataFrame(SCORE_vgg19, columns=["Image", "Accuracy", "F1", "Jaccard", "Recall", "Precision"])
df.to_csv("files/score_vgg19.csv", index=None)

In [ ]:
scores = pd.read_csv("/kaggle/working/files/score_vgg19.csv")
scores.head(3)

In [ ]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

create_dir("files_resnet50")
create_dir("results_resnet50")

model_path_resnet50 = os.path.join("files_resnet50", "model_resnet50.h5")
csv_path_resnet50 = os.path.join("files_resnet50", "log_resnet50.csv")

callbacks_resnet50 = [
    ModelCheckpoint(model_path_resnet50, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
    CSVLogger(csv_path_resnet50),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False),
]

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Conv2DTranspose, Concatenate, Conv2D
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import numpy as np

# Define the U-Net model with ResNet50 encoder
def build_resnet50(input_shape):
    # Encoder: load ResNet50 with pre-trained ImageNet weights
    encoder = ResNet50(include_top=False, weights='imagenet', input_tensor=Input(shape=input_shape))

    # Set the encoder layers to not be trainable
    for layer in encoder.layers:
        layer.trainable = False

    # Skip connections from specific layers in ResNet50
    skip_connections = ['conv2_block3_out', 'conv3_block4_out', 'conv4_block6_out', 'conv5_block3_out']
    encoder_outputs = [encoder.get_layer(name).output for name in skip_connections]

    # Decoder
    x = encoder.output
    for i in range(len(encoder_outputs)):
        num_filters = 256 // (2 ** i)
        x = Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(x)
        skip_output = encoder_outputs[-(i + 1)]
        skip_output = tf.image.resize(skip_output, tf.shape(x)[1:3], method='nearest')
        x = Concatenate()([x, skip_output])
        x = conv_block(x, num_filters)

    # Final Conv2DTranspose layer to upscale to the original image size
    x = Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(x)
    outputs = Conv2D(1, (1, 1), activation='sigmoid', padding='same')(x)

    # Create the model
    model = Model(inputs=encoder.input, outputs=outputs)
    return model

# Initialize the model
resnet50_model = build_resnet50((H, W, 3))

# Compile the model
resnet50_model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coef, 'accuracy', iou])

# Print the model summary to check the final model architecture
resnet50_model.summary()

# Train the model using the same datasets and callbacks you've defined
history_resnet50 = resnet50_model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    callbacks=callbacks_resnet50
)

# Save training history
np.save('history_resnet50.npy', history_resnet50.history)

# Evaluate the model on your test dataset
resnet50_test_metrics = resnet50_model.evaluate(test_dataset)
print(f"ResNet50 Test Loss: {resnet50_test_metrics[0]}")
print(f"ResNet50 Test Dice Coefficient: {resnet50_test_metrics[1]}")
print(f"ResNet50 Test Accuracy: {resnet50_test_metrics[2]}")
print(f"ResNet50 Test IoU: {resnet50_test_metrics[3]}")

# Save and display the test metrics
with open('resnet50_test_metrics.txt', 'w') as file:
    file.write(str(resnet50_test_metrics))

In [ ]:
   """Prediction and Evaluation"""
SCORE_resnet50= []
for x, y in tqdm(zip(test_x, test_y), total=len(test_y)):
    """ Extracting the name """
    name = x.split("/")[-1]
    
    """ Reading the image """
    image = cv2.imread(x, cv2.IMREAD_COLOR) ## [H, w, 3]
    image = cv2.resize(image, (W, H))       ## [H, w, 3]
    x = image/255.0                         ## [H, w, 3]
    x = np.expand_dims(x, axis=0)           ## [1, H, w, 3]

    """ Reading the mask """
    mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (W, H))

    """ Prediction """
    y_pred = resnet50_model.predict(x, verbose=0)[0]
    y_pred = np.squeeze(y_pred, axis=-1)
    y_pred = y_pred >= 0.5
    y_pred = y_pred.astype(np.int32)

    """ Saving the prediction """
    save_image_path = os.path.join("results_resnet50 ", name)
    save_results(image, mask, y_pred, save_image_path)

    """ Flatten the array """
    mask = mask/255.0
    mask = (mask > 0.5).astype(np.int32).flatten()
    y_pred = y_pred.flatten()

    """ Calculating the metrics values """
    accuracy = accuracy_score(mask, y_pred)
    f1_value = f1_score(mask, y_pred, labels=[0, 1], average="binary")
    jac_value = jaccard_score(mask, y_pred, labels=[0, 1], average="binary")
    recall_value = recall_score(mask, y_pred, labels=[0, 1], average="binary", zero_division=0)
    precision_value = precision_score(mask, y_pred, labels=[0, 1], average="binary", zero_division=0)
    SCORE_resnet50.append([name, accuracy, f1_value, jac_value, recall_value, precision_value])

""" Metrics values """
score_resnet50 = [s[1:]for s in SCORE_resnet50]
score_resnet50 = np.mean(score_resnet50, axis=0)
print(f"Accuracy: {score_resnet50[0]:0.5f}")
print(f"F1: {score_resnet50[1]:0.5f}")
print(f"Jaccard: {score_resnet50[2]:0.5f}")
print(f"Recall: {score_resnet50[3]:0.5f}")
print(f"Precision: {score_resnet50[4]:0.5f}")

df = pd.DataFrame(SCORE_resnet50, columns=["Image", "Accuracy", "F1", "Jaccard", "Recall", "Precision"])
df.to_csv("files/score_resnet50.csv", index=None)

In [ ]:
scores = pd.read_csv("/kaggle/working/files/score_resnet50.csv")
scores.head(3)

In [ ]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

create_dir("files_mobilenetv2")
create_dir("results_mobilenetv2")

model_path_mobilenetv2 = os.path.join("files_mobilenetv2", "model_mobilenetv2.h5")
csv_path_mobilenetv2 = os.path.join("files_mobilenetv2", "log_mobilenetv2.csv")

callbacks_mobilenetv2 = [
    ModelCheckpoint(model_path_mobilenetv2, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
    CSVLogger(csv_path_mobilenetv2),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False),
]

In [ ]:
def build_mobilenetv2(input_shape):
    # Encoder: load MobileNetV2 with pre-trained ImageNet weights
    encoder = MobileNetV2(include_top=False, weights='imagenet', input_tensor=Input(shape=input_shape))

    # Set the encoder layers to not be trainable
    for layer in encoder.layers:
        layer.trainable = False

    # Skip connections from specific layers in MobileNetV2
    skip_connections = ['block_1_expand_relu', 'block_3_expand_relu', 'block_6_expand_relu', 'block_13_expand_relu']
    encoder_outputs = [encoder.get_layer(name).output for name in skip_connections]

    # Decoder
    x = encoder.output
    num_filters = 512
    for skip_output in reversed(encoder_outputs):
        x = Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(x)
        skip_output = tf.image.resize(skip_output, tf.shape(x)[1:3], method='nearest')
        x = Concatenate()([x, skip_output])
        x = conv_block(x, num_filters)
        num_filters //= 2

    # Ensure the final layer has the same size as the input
    x = Conv2DTranspose(input_shape[-1], (2, 2), strides=(2, 2), padding='same')(x)
    outputs = Conv2D(1, (1, 1), activation='sigmoid', padding='same')(x)

    # Create the model
    model = Model(inputs=encoder.input, outputs=outputs)
    return model

In [ ]:
# Initialize the model
mobilenetv2_model = build_mobilenetv2((H, W, 3))

# Compile the model
mobilenetv2_model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coef, 'accuracy', iou])

# Print the model summary to check the final model architecture
mobilenetv2_model.summary()

# Train the model using the same datasets and callbacks you've defined
history_mobilenetv2 = mobilenetv2_model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    callbacks=callbacks_mobilenetv2
)

# Save training history
np.save('history_mobilenetv2.npy', history_mobilenetv2.history)

# Evaluate the model on your test dataset
mobilenetv2_test_metrics = mobilenetv2_model.evaluate(test_dataset)

# Save and display the test metrics
with open('mobilenetv2_test_metrics.txt', 'w') as file:
    file.write(str(mobilenetv2_test_metrics))

print(f"MobileNetV2 Test Loss: {mobilenetv2_test_metrics[0]}")
print(f"MobileNetV2 Test Dice Coefficient: {mobilenetv2_test_metrics[1]}")
print(f"MobileNetV2 Test Accuracy: {mobilenetv2_test_metrics[2]}")
print(f"MobileNetV2 Test IoU: {mobilenetv2_test_metrics[3]}")

In [ ]:
   """Prediction and Evaluation"""
SCORE_mobilenetv2= []
for x, y in tqdm(zip(test_x, test_y), total=len(test_y)):
    """ Extracting the name """
    name = x.split("/")[-1]


    """ Reading the image """
    image = cv2.imread(x, cv2.IMREAD_COLOR) ## [H, w, 3]
    image = cv2.resize(image, (W, H))       ## [H, w, 3]
    x = image/255.0                         ## [H, w, 3]
    x = np.expand_dims(x, axis=0)           ## [1, H, w, 3]

    """ Reading the mask """
    mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (W, H))

    """ Prediction """
    y_pred = mobilenetv2_model.predict(x, verbose=0)[0]
    y_pred = np.squeeze(y_pred, axis=-1)
    y_pred = y_pred >= 0.5
    y_pred = y_pred.astype(np.int32)

    """ Saving the prediction """
    save_image_path = os.path.join("results_resnet50 ", name)
    save_results(image, mask, y_pred, save_image_path)

    """ Flatten the array """
    mask = mask/255.0
    mask = (mask > 0.5).astype(np.int32).flatten()
    y_pred = y_pred.flatten()

    """ Calculating the metrics values """
    accuracy = accuracy_score(mask, y_pred)
    f1_value = f1_score(mask, y_pred, labels=[0, 1], average="binary")
    jac_value = jaccard_score(mask, y_pred, labels=[0, 1], average="binary")
    recall_value = recall_score(mask, y_pred, labels=[0, 1], average="binary", zero_division=0)
    precision_value = precision_score(mask, y_pred, labels=[0, 1], average="binary", zero_division=0)
    SCORE_mobilenetv2.append([name, accuracy, f1_value, jac_value, recall_value, precision_value])

""" Metrics values """
score_mobilenetv2 = [s[1:]for s in SCORE_mobilenetv2]
score_mobilenetv2 = np.mean(score_mobilenetv2, axis=0)
print(f"Accuracy: {score_mobilenetv2[0]:0.5f}")
print(f"F1: {score_mobilenetv2[1]:0.5f}")
print(f"Jaccard: {score_mobilenetv2[2]:0.5f}")
print(f"Recall: {score_mobilenetv2[3]:0.5f}")
print(f"Precision: {score_mobilenetv2[4]:0.5f}")

df = pd.DataFrame(SCORE_resnet50, columns=["Image", "Accuracy", "F1", "Jaccard", "Recall", "Precision"])
df.to_csv("files/score_mobilenetv2.csv", index=None)

In [ ]:
scores = pd.read_csv("/kaggle/working/files/score_mobilenetv2.csv")
scores.head(3)

In [ ]:
import pandas as pd

# Paths to the metrics files
metrics_files = {
    'UNet': '/kaggle/working/unet_test_metrics.txt',
    'VGG16': '/kaggle/working/vgg16_test_metrics.txt',
    'VGG19': '/kaggle/working/vgg19_test_metrics.txt',
    'ResNet50': '/kaggle/working/resnet50_test_metrics.txt',
    'MobileNetV2': '/kaggle/working/mobilenetv2_test_metrics.txt'
}

# Read the metrics into a DataFrame
metrics_data = []
for model, filepath in metrics_files.items():
    with open(filepath, 'r') as file:
        # Assuming that each file has a single line with the metrics in order
        metrics = file.readline().strip().strip('[]').split(',')
        metrics_data.append([model] + [float(metric.strip()) for metric in metrics])

# Assuming the order is Loss, Dice Coefficient, Accuracy, IoU
df_metrics = pd.DataFrame(metrics_data, columns=['Model', 'Test Loss', 'Dice Coefficient', 'Accuracy', 'IoU'])

print(df_metrics)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming df_metrics is your DataFrame
# with columns 'Model', 'Test Loss', 'Dice Coefficient', 'Accuracy', 'IoU'

# Set the style
sns.set(style="whitegrid")

# Melt the DataFrame to make it suitable for sns.barplot
df_melted = df_metrics.melt(id_vars='Model', var_name='Metric', value_name='Score')

# Set the size of the figure
plt.figure(figsize=(14, 8))

# Create the barplot
ax = sns.barplot(x='Metric', y='Score', hue='Model', data=df_melted)

# Customize the plot to make it more appealing
ax.set_title('Comparative Performance Metrics Across Models', fontsize=18, pad=20)
ax.set_xlabel('Metric', fontsize=14, labelpad=15)
ax.set_ylabel('Score', fontsize=14, labelpad=15)
ax.tick_params(labelsize=12)

# Add value labels on top of the bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.2f', label_type='edge', padding=3, fontsize=10)

# Increase the bottom margin to accommodate model names
plt.subplots_adjust(bottom=0.15)

# Move the legend outside of the plot
plt.legend(title='Model', title_fontsize='13', loc='upper left', bbox_to_anchor=(1, 1), fontsize=12)

# Display the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Paths to the CSV files for each model
model_paths = {
    'UNet': '/kaggle/working/files/log.csv',
    'VGG16': '/kaggle/working/files_vgg16/log_vgg16.csv',
    'VGG19': '/kaggle/working/files_vgg19/log_vgg19.csv',
    'ResNet50': '/kaggle/working/files_resnet50/log_resnet50.csv',
    'MobileNetV2': '/kaggle/working/files_mobilenetv2/log_mobilenetv2.csv'
}

# Metrics to plot
metrics = ['accuracy', 'dice_coef', 'iou', 'loss']

# Set the aesthetic style of the plots
sns.set_style("whitegrid")

# Plot training metrics for each model
for model_name, file_path in model_paths.items():
    # Load the CSV file into a DataFrame
    df_metrics = pd.read_csv(file_path)

    # Create a line plot for each metric
    fig, axs = plt.subplots(1, 4, figsize=(20, 5), sharex=True)
    fig.suptitle(f'{model_name} Training Progression', fontsize=16)

    for i, metric in enumerate(metrics):
        axs[i].plot(df_metrics['epoch'], df_metrics[metric], label=f'Train {metric}')
        axs[i].plot(df_metrics['epoch'], df_metrics[f'val_{metric}'], label=f'Val {metric}')
        axs[i].set_title(f'{metric.capitalize()} Over Epochs')
        axs[i].set_xlabel('Epoch')
        axs[i].set_ylabel(metric.capitalize())
        axs[i].legend()

    # Show the plot with a tight layout
    plt.tight_layout()
    plt.subplots_adjust(top=0.88)  # Adjust the top to fit the suptitle
    plt.show()

In [ ]:
#saving above plot
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from IPython.display import FileLink

# Paths to the CSV files for each model
model_paths = {
    'UNet': '/kaggle/working/files/log.csv',
    'VGG16': '/kaggle/working/files_vgg16/log_vgg16.csv',
    'VGG19': '/kaggle/working/files_vgg19/log_vgg19.csv',
    'ResNet50': '/kaggle/working/files_resnet50/log_resnet50.csv',
    'MobileNetV2': '/kaggle/working/files_mobilenetv2/log_mobilenetv2.csv'
}

# Metrics to plot
metrics = ['accuracy', 'dice_coef', 'iou', 'loss']

# Set the aesthetic style of the plots
sns.set_style("whitegrid")

# Plot training metrics for each model
for model_name, file_path in model_paths.items():
    # Load the CSV file into a DataFrame
    df_metrics = pd.read_csv(file_path)

    # Create a line plot for each metric
    fig, axs = plt.subplots(1, 4, figsize=(20, 5), sharex=True)
    fig.suptitle(f'{model_name} Training Progression', fontsize=16)

    for i, metric in enumerate(metrics):
        axs[i].plot(df_metrics['epoch'], df_metrics[metric], label=f'Train {metric}')
        axs[i].plot(df_metrics['epoch'], df_metrics[f'val_{metric}'], label=f'Val {metric}')
        axs[i].set_title(f'{metric.capitalize()} Over Epochs')
        axs[i].set_xlabel('Epoch')
        axs[i].set_ylabel(metric.capitalize())
        axs[i].legend()

    # Save the plot to a file
    plot_filename = f'/kaggle/working/{model_name.lower()}_training_progression.png'
    plt.savefig(plot_filename)
    plt.close(fig)  # Close the figure to avoid displaying it in the notebook

    # Create a link to download the saved plot
    display(FileLink(plot_filename))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# File paths
score_files = {
    'U-Net': '/kaggle/working/files/score.csv',
    'VGG16': '/kaggle/working/files/score_vgg16.csv',
    'VGG19': '/kaggle/working/files/score_vgg19.csv',
    'ResNet50': '/kaggle/working/files/score_resnet50.csv',
    'MobileNetV2': '/kaggle/working/files/score_mobilenetv2.csv'
}

# Read and combine the data
all_scores = []
for model, file_path in score_files.items():
    df = pd.read_csv(file_path)
    df['Model'] = model  # Add a column for the model name
    all_scores.append(df)

combined_scores = pd.concat(all_scores)

# Reshape the data
melted_scores = combined_scores.melt(id_vars='Model', 
                                     value_vars=['F1', 'Jaccard', 'Recall', 'Precision'], 
                                     var_name='Metric', 
                                     value_name='Score')

# Set the style
sns.set(style="whitegrid")

# Set the size of the figure
plt.figure(figsize=(14, 8))

# Create the barplot
ax = sns.barplot(x='Metric', y='Score', hue='Model', data=melted_scores, errorbar=None)

# Customize the plot to make it more appealing
ax.set_title('Comparative Evaluation Metrics Across Models', fontsize=18, pad=20)
ax.set_xlabel('Metric', fontsize=14, labelpad=15)
ax.set_ylabel('Score', fontsize=14, labelpad=15)
ax.tick_params(labelsize=12)

# Add value labels on top of the bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.2f', label_type='edge', padding=3, fontsize=10)

# Increase the bottom margin to accommodate model names
plt.subplots_adjust(bottom=0.15)

# Move the legend outside of the plot
plt.legend(title='Model', title_fontsize='13', loc='upper left', bbox_to_anchor=(1, 1), fontsize=12)

# Display the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming df_metrics is your DataFrame
# with columns 'Model', 'Test Loss', 'Dice Coefficient', 'Accuracy', 'IoU'

# Set the style
sns.set(style="whitegrid", palette="pastel")

# Melt the DataFrame to make it suitable for sns.barplot
df_melted = df_metrics.melt(id_vars='Model', var_name='Metric', value_name='Score')

# Set the size of the figure
plt.figure(figsize=(14, 8))

# Create the barplot
ax = sns.barplot(x='Metric', y='Score', hue='Model', data=df_melted)

# Customize the plot to make it more appealing
ax.set_title('Comparative Performance Metrics Across Models', fontsize=18, pad=20)
ax.set_xlabel('Metric', fontsize=14, labelpad=15)
ax.set_ylabel('Score', fontsize=14, labelpad=15)
ax.tick_params(labelsize=12)

# Add value labels on top of the bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.2f', label_type='edge', padding=3, fontsize=10)

# Increase the bottom margin to accommodate model names
plt.subplots_adjust(bottom=0.15)

# Move the legend outside of the plot
plt.legend(title='Model', title_fontsize='13', loc='upper left', bbox_to_anchor=(1, 1), fontsize=12)

# Display the plot
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# File paths
score_files = {
    'U-Net': '/kaggle/working/files/score.csv',
    'VGG16': '/kaggle/working/files/score_vgg16.csv',
    'VGG19': '/kaggle/working/files/score_vgg19.csv',
    'ResNet50': '/kaggle/working/files/score_resnet50.csv',
    'MobileNetV2': '/kaggle/working/files/score_mobilenetv2.csv'
}

# Read and combine the data
all_scores = []
for model, file_path in score_files.items():
    df = pd.read_csv(file_path)
    df['Model'] = model  # Add a column for the model name
    all_scores.append(df)

combined_scores = pd.concat(all_scores)

# Reshape the data
melted_scores = combined_scores.melt(id_vars=['Model', 'Image'], 
                                     value_vars=['F1', 'Jaccard', 'Recall', 'Precision'], 
                                     var_name='Metric', 
                                     value_name='Score')

# Plotting
plt.figure(figsize=(12, 6))
bar_plot = sns.barplot(x='Model', y='Score', hue='Metric', data=melted_scores,errorbar=None)

# Adding labels
for p in bar_plot.patches:
    bar_plot.annotate(format(p.get_height(), '.2f'), 
                      (p.get_x() + p.get_width() / 2., p.get_height()), 
                      ha = 'center', va = 'center', 
                      xytext = (0, 9), 
                      textcoords = 'offset points')

plt.title('Performance Metrics for Different Models')
plt.xlabel('Model')
plt.ylabel('Score')
plt.legend(title='Metric')
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
bar_plot = sns.barplot(x='Model', y='Score', hue='Metric', data=melted_scores, ci=None)  # Added ci=None to remove error bars

# The rest of your code remains the same
...


In [ ]:
# Plot the metrics
ax = metrics_df.plot(kind='bar', figsize=(10, 7), width=0.8)
plt.title('Test Metrics Comparison Across Models')
plt.ylabel('Value')
plt.xticks(rotation=45)
plt.legend(title='Metric', bbox_to_anchor=(1.05, 1), loc='upper left')  # Moves the legend outside of the plot
plt.tight_layout(rect=[0, 0, 0.85, 1])  # Adjust the rect to prevent the legend from cutting off

# Adding labels on top of each bar
for p in ax.patches:
    # Calculate index for x-tick labels
    index = int(p.get_x() + p.get_width() / 2.0) // len(numeric_cols)
    model_name = ax.get_xticklabels()[index].get_text()
    
    # Set different offsets for labels, especially for the last set of bars
    offset = (0, 10) if 'MobileNetV2' not in model_name else (0, 30)
    ax.annotate(f"{p.get_height():.2f}", 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='center', 
                xytext=offset, 
                textcoords='offset points')

plt.show()
